In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Punta Arenas,CL,2021-09-30 22:18:08,-53.1500,-70.9167,44.71,49,0,20.71
1,1,Bluff,NZ,2021-09-30 22:18:09,-46.6000,168.3333,52.81,80,67,21.00
2,2,Nikolskoye,RU,2021-09-30 22:14:07,59.7035,30.7861,44.49,94,17,4.65
3,3,Geraldton,AU,2021-09-30 22:16:56,-28.7667,114.6000,64.81,94,40,10.36
4,4,Lufilufi,WS,2021-09-30 22:18:11,-13.8745,-171.5986,91.18,77,68,23.80


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
8,8,Hilo,US,2021-09-30 22:18:14,19.7297,-155.0900,77.25,94,90,3.44
13,13,Lethem,GY,2021-09-30 22:18:20,3.3803,-59.7968,86.20,53,86,3.78
20,20,Vaini,TO,2021-09-30 22:18:25,-21.2000,-175.2000,77.16,83,75,6.91
26,26,Itaituba,BR,2021-09-30 22:18:31,-4.2761,-55.9836,89.76,66,20,1.50
28,28,Avarua,CK,2021-09-30 22:15:16,-21.2078,-159.7750,80.65,74,20,13.80
34,34,Tautira,PF,2021-09-30 22:18:37,-17.7333,-149.1500,80.62,77,23,16.35
37,37,Butaritari,KI,2021-09-30 22:15:16,3.0707,172.7902,81.57,77,99,12.66
54,54,Kapaa,US,2021-09-30 22:18:52,22.0752,-159.3190,84.18,69,40,5.99
55,55,Mandali,IQ,2021-09-30 22:18:53,33.7499,45.5541,81.70,20,0,8.21
59,59,Kirando,TZ,2021-09-30 22:18:56,-7.4167,30.6000,81.72,55,99,4.54


In [10]:
preferred_cities_df.count()

City_ID       162
City          162
Country       162
Date          162
Lat           162
Lng           162
Max Temp      162
Humidity      162
Cloudiness    162
Wind Speed    162
dtype: int64

In [22]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Hilo,US,77.25,19.7297,-155.0900,
13,Lethem,GY,86.20,3.3803,-59.7968,
20,Vaini,TO,77.16,-21.2000,-175.2000,
26,Itaituba,BR,89.76,-4.2761,-55.9836,
28,Avarua,CK,80.65,-21.2078,-159.7750,
34,Tautira,PF,80.62,-17.7333,-149.1500,
37,Butaritari,KI,81.57,3.0707,172.7902,
54,Kapaa,US,84.18,22.0752,-159.3190,
55,Mandali,IQ,81.70,33.7499,45.5541,
59,Kirando,TZ,81.72,-7.4167,30.6000,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [ ]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [28]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Hilo,US,77.25,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Lethem,GY,86.20,3.3803,-59.7968,Savannah Inn
20,Vaini,TO,77.16,-21.2000,-175.2000,Keleti Beach Resort
26,Itaituba,BR,89.76,-4.2761,-55.9836,Hotel Campos
28,Avarua,CK,80.65,-21.2078,-159.7750,Paradise Inn


In [30]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [33]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [34]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))